## Steps

#### Load observations, greenspaces, ocean and continetal waters
#### Exclude the greenspaces within ocean and continental waters (key: nature_reserve)
#### Exclude overlapping geometries 
#### Export files as .shp

In [1]:
#import libraries
import pandas as pd 
import numpy as np
from glob2 import glob
import geopandas as gpd 
import osmnx as ox

In [2]:
#get California as gdf to check anomalies. Points outside California will be excluded
ca=ox.geocode_to_gdf('California')

In [3]:
#load observations
data_folder = '../data/observations_final'
df = pd.concat([pd.read_csv(f).assign(challenge=f.replace('.csv','')) for f in glob(data_folder+'/CNC_San_Francisco_*.csv')])

In [4]:
df.shape

(139040, 39)

In [5]:
#exclude anomalies
df=df[df['latitude'].le(ca['bbox_north'].iloc[0]) \
                  & df['latitude'].ge(ca['bbox_south'].iloc[0]) \
                  & df['longitude'].le(ca['bbox_east'].iloc[0]) \
                  & df['longitude'].ge(ca['bbox_west'].iloc[0])]

In [6]:
#create year 
df['year']=df['time_observed_at'].str[0:4].astype('int64')

In [7]:
df.shape

(139038, 40)

In [8]:
#load greenspaces
#http://download.geofabrik.de/north-america/us/california/socal.html
landuse=gpd.read_file('../data/norcal-latest-free/gis_osm_landuse_a_free_1.shp')

In [9]:
landuse.shape

(293241, 5)

In [10]:
landuse.head()

,osm_id,code,fclass,name,geometry
0,4786596,7202,park,Klein Park,"POLYGON ((-122.10458 37.40042, -122.10457 37.4..."
1,7459901,7202,park,Buena Vista Park,"POLYGON ((-122.44378 37.76757, -122.44358 37.7..."
2,7459901,7201,forest,Buena Vista Park,"POLYGON ((-122.44378 37.76757, -122.44358 37.7..."
3,7805676,7202,park,People's Park,"POLYGON ((-122.25798 37.86603, -122.25643 37.8..."
4,8948101,7202,park,Babbs Creek Park Preserve,"POLYGON ((-121.57734 36.98635, -121.57734 36.9..."


In [11]:
#define function to get coordinates using the centroids
def getXY(pt):
    return (pt.x, pt.y)

In [12]:
#get coordinates
landuse['centroid'] = landuse['geometry'].centroid
landuse['lon'], landuse['lat'] = [list(t) for t in zip(*map(getXY, landuse.centroid))]

C:\Users\acmar\miniconda3\envs\cs\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  
C:\Users\acmar\miniconda3\envs\cs\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
landuse.head()

,osm_id,code,fclass,name,geometry,centroid,lon,lat
0,4786596,7202,park,Klein Park,"POLYGON ((-122.10458 37.40042, -122.10457 37.4...",POINT (-122.10394 37.40072),-122.103938,37.400718
1,7459901,7202,park,Buena Vista Park,"POLYGON ((-122.44378 37.76757, -122.44358 37.7...",POINT (-122.44139 37.76854),-122.441393,37.768543
2,7459901,7201,forest,Buena Vista Park,"POLYGON ((-122.44378 37.76757, -122.44358 37.7...",POINT (-122.44139 37.76854),-122.441393,37.768543
3,7805676,7202,park,People's Park,"POLYGON ((-122.25798 37.86603, -122.25643 37.8...",POINT (-122.25713 37.86576),-122.257131,37.865758
4,8948101,7202,park,Babbs Creek Park Preserve,"POLYGON ((-121.57734 36.98635, -121.57734 36.9...",POINT (-121.57547 36.98769),-121.575468,36.987693


In [14]:
#limit greenspaces with observations as boundaries (lat)
l1 = landuse[landuse['lat'].ge(df.latitude.min()) & landuse['lat'].le(df.latitude.max())]

In [15]:
#limit greenspaces with observations as boundaries (lon)
l2 = l1[l1['lon'].ge(df.longitude.min()) & l1['lon'].le(df.longitude.max())]

In [16]:
landuse_clean=l2

In [17]:
landuse.shape, landuse_clean.shape

((293241, 8), (58198, 8))

In [19]:
green=landuse_clean[landuse_clean['fclass'].eq('grass') | 
        landuse_clean['fclass'].eq('park') | 
        landuse_clean['fclass'].eq('forest') | 
        landuse_clean['fclass'].eq('scrub') |
        landuse_clean['fclass'].eq('vineyard') | 
        landuse_clean['fclass'].eq('meadow') |
        landuse_clean['fclass'].eq('orchard') |
        landuse_clean['fclass'].eq('heath') |
        landuse_clean['fclass'].eq('recreation_ground') |
        landuse_clean['fclass'].eq('allotments') | 
        landuse_clean['fclass'].eq('nature_reserve')]

In [20]:
green.shape

(34910, 8)

In [21]:
green.head()

,osm_id,code,fclass,name,geometry,centroid,lon,lat
0,4786596,7202,park,Klein Park,"POLYGON ((-122.10458 37.40042, -122.10457 37.4...",POINT (-122.10394 37.40072),-122.103938,37.400718
1,7459901,7202,park,Buena Vista Park,"POLYGON ((-122.44378 37.76757, -122.44358 37.7...",POINT (-122.44139 37.76854),-122.441393,37.768543
2,7459901,7201,forest,Buena Vista Park,"POLYGON ((-122.44378 37.76757, -122.44358 37.7...",POINT (-122.44139 37.76854),-122.441393,37.768543
3,7805676,7202,park,People's Park,"POLYGON ((-122.25798 37.86603, -122.25643 37.8...",POINT (-122.25713 37.86576),-122.257131,37.865758
5,10321607,7202,park,César Chávez Park,"POLYGON ((-122.32481 37.87452, -122.32466 37.8...",POINT (-122.31959 37.87253),-122.319586,37.872531


In [22]:
# drop centroid
green_clean = green.drop(['centroid'], axis=1)

In [23]:
green_clean.head()

,osm_id,code,fclass,name,geometry,lon,lat
0,4786596,7202,park,Klein Park,"POLYGON ((-122.10458 37.40042, -122.10457 37.4...",-122.103938,37.400718
1,7459901,7202,park,Buena Vista Park,"POLYGON ((-122.44378 37.76757, -122.44358 37.7...",-122.441393,37.768543
2,7459901,7201,forest,Buena Vista Park,"POLYGON ((-122.44378 37.76757, -122.44358 37.7...",-122.441393,37.768543
3,7805676,7202,park,People's Park,"POLYGON ((-122.25798 37.86603, -122.25643 37.8...",-122.257131,37.865758
5,10321607,7202,park,César Chávez Park,"POLYGON ((-122.32481 37.87452, -122.32466 37.8...",-122.319586,37.872531


In [24]:
#ocean
#https://osmdata.openstreetmap.de/data/water-polygons.html
water=gpd.read_file('../data/water-polygons-split-4326/water-polygons-split-4326/water_polygons.shp')

In [25]:
water.shape

(53282, 3)

In [26]:
water.head()

,x,y,geometry
0,1,41,"POLYGON ((0.99933 40.99950, 0.99933 41.04319, ..."
1,-11,-72,"POLYGON ((-11.00158 -71.04396, -11.00158 -71.0..."
2,-11,-72,"POLYGON ((-10.81949 -70.99950, -10.75741 -70.9..."
3,148,-11,"POLYGON ((147.99949 -11.00050, 147.99949 -10.1..."
4,-25,81,"POLYGON ((-25.00338 81.58330, -25.00338 81.720..."


In [27]:
water['centroid'] = water['geometry'].centroid
water['lon'], water['lat'] = [list(t) for t in zip(*map(getXY, water.centroid))]

C:\Users\acmar\miniconda3\envs\cs\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.
C:\Users\acmar\miniconda3\envs\cs\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


In [28]:
water.head()

,x,y,geometry,centroid,lon,lat
0,1,41,"POLYGON ((0.99933 40.99950, 0.99933 41.04319, ...",POINT (1.61701 41.09437),1.617007,41.094369
1,-11,-72,"POLYGON ((-11.00158 -71.04396, -11.00158 -71.0...",POINT (-10.97495 -71.01612),-10.974950,-71.016119
2,-11,-72,"POLYGON ((-10.81949 -70.99950, -10.75741 -70.9...",POINT (-10.75569 -71.00237),-10.755689,-71.002365
3,148,-11,"POLYGON ((147.99949 -11.00050, 147.99949 -10.1...",POINT (148.48457 -10.58733),148.484570,-10.587332
4,-25,81,"POLYGON ((-25.00338 81.58330, -25.00338 81.720...",POINT (-24.42133 81.79519),-24.421329,81.795188


In [29]:
w1 = water[water['lat'].ge(df.latitude.min()) & water['lat'].le(df.latitude.max())]

In [30]:
w2 = w1[w1['lon'].ge(df.longitude.min()) & w1['lon'].le(df.longitude.max())]

In [31]:
water_clean=w2

In [32]:
water_clean.shape

(12, 6)

In [33]:
water_clean.head()

,x,y,geometry,centroid,lon,lat
3509,-123,37,"POLYGON ((-123.00063 36.99950, -123.00063 37.6...",POINT (-122.66656 37.46817),-122.666558,37.468167
3510,-123,37,"POLYGON ((-121.99937 37.48335, -121.99937 37.4...",POINT (-121.99975 37.48213),-121.999750,37.482129
5175,-122,37,"POLYGON ((-122.00063 37.46652, -122.00063 37.4...",POINT (-121.97486 37.46015),-121.974856,37.460154
5176,-122,37,"POLYGON ((-122.00063 37.47850, -122.00063 37.4...",POINT (-121.99743 37.48085),-121.997432,37.480849
8313,-124,38,"POLYGON ((-124.00064 37.99950, -124.00064 39.0...",POINT (-123.59113 38.39751),-123.591133,38.397513


In [34]:
# drop centroid
water_clean_2 = water_clean.drop(['centroid'], axis=1)

In [35]:
water_clean_2

,x,y,geometry,lon,lat
3509,-123,37,"POLYGON ((-123.00063 36.99950, -123.00063 37.6...",-122.666558,37.468167
3510,-123,37,"POLYGON ((-121.99937 37.48335, -121.99937 37.4...",-121.999750,37.482129
5175,-122,37,"POLYGON ((-122.00063 37.46652, -122.00063 37.4...",-121.974856,37.460154
5176,-122,37,"POLYGON ((-122.00063 37.47850, -122.00063 37.4...",-121.997432,37.480849
8313,-124,38,"POLYGON ((-124.00064 37.99950, -124.00064 39.0...",-123.591133,38.397513
9163,-124,37,"POLYGON ((-124.00063 36.99950, -124.00063 38.0...",-123.500088,37.499924
10598,-122,38,"POLYGON ((-122.00064 38.05396, -122.00064 38.0...",-121.841645,38.052843
10599,-122,38,"POLYGON ((-122.00064 38.10270, -122.00064 38.1...",-121.994687,38.118976
12251,-123,38,"POLYGON ((-123.00064 38.02520, -123.00064 38.2...",-122.961141,38.161426
12252,-123,38,"POLYGON ((-123.00064 38.29697, -123.00064 38.2...",-122.979821,38.309495


In [36]:
#http://download.geofabrik.de/north-america/us/california/norcal.html
water_cont=gpd.read_file('../data/norcal-latest-free/gis_osm_water_a_free_1.shp')

In [37]:
water_cont.shape

(59994, 5)

In [38]:
water_cont['centroid'] = water_cont['geometry'].centroid
water_cont['lon'], water_cont['lat'] = [list(t) for t in zip(*map(getXY, water_cont.centroid))]

C:\Users\acmar\miniconda3\envs\cs\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.
C:\Users\acmar\miniconda3\envs\cs\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


In [39]:
wc1 = water_cont[water_cont['lat'].ge(df.latitude.min()) & water_cont['lat'].le(df.latitude.max())]

In [40]:
wc2 = wc1[wc1['lon'].ge(df.longitude.min()) & wc1['lon'].le(df.longitude.max())]

In [41]:
water_cont_clean=wc2

In [42]:
water_cont.shape, water_cont_clean.shape

((59994, 8), (10380, 8))

In [43]:
water_cont_clean.head()

,osm_id,code,fclass,name,geometry,centroid,lon,lat
0,4896447,8202,riverbank,None,"POLYGON ((-122.70267 38.24746, -122.70247 38.2...",POINT (-122.63831 38.24693),-122.638309,38.246929
1,4896517,8202,riverbank,None,"POLYGON ((-122.57006 38.19333, -122.56941 38.1...",POINT (-122.55826 38.19165),-122.558258,38.191645
2,4896562,8200,water,None,"POLYGON ((-122.24904 37.72559, -122.24875 37.7...",POINT (-122.24667 37.72587),-122.246675,37.725873
3,4896904,8202,riverbank,None,"POLYGON ((-122.57010 38.17702, -122.56991 38.1...",POINT (-122.55849 38.16819),-122.558491,38.168191
4,4896933,8200,water,None,"POLYGON ((-122.01848 38.22699, -122.01840 38.2...",POINT (-122.01667 38.22713),-122.016672,38.227131


In [44]:
water_cont_clean_2=water_cont_clean.drop(['centroid'], axis=1)

In [45]:
green_clean_difference=gpd.overlay(green_clean, water_clean_2, how='difference')

In [46]:
green_clean.shape, green_clean_difference.shape

((34910, 7), (34903, 7))

In [47]:
green_clean_difference_2=gpd.overlay(green_clean_difference, water_cont_clean_2, how='difference')

In [48]:
green=green_clean_difference_2

In [49]:
geom = green.geometry.unary_union

In [50]:
green_unique = gpd.GeoDataFrame(geometry=[geom],crs='epsg:4326')

In [51]:
green_unique.head()

,geometry
0,"MULTIPOLYGON (((-122.10455 36.96578, -122.1044..."


In [52]:
green_unique.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [53]:
green_unique = green_unique.explode().reset_index(drop=True)

In [54]:
green_unique.head()

,geometry
0,"POLYGON ((-122.10455 36.96578, -122.10448 36.9..."
1,"POLYGON ((-121.96565 37.03789, -121.96560 37.0..."
2,"POLYGON ((-122.23961 37.05038, -122.23919 37.0..."
3,"POLYGON ((-121.65561 37.04986, -121.65529 37.0..."
4,"POLYGON ((-122.26762 37.07843, -122.26760 37.0..."


In [55]:
green_unique.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [56]:
green_unique.shape

(23264, 1)

In [57]:
geom = water_cont_clean_2.geometry.unary_union

In [58]:
water_cont_clean_2_unique = gpd.GeoDataFrame(geometry=[geom],crs='epsg:4326')

In [59]:
water_cont_clean_2_unique.head()

,geometry
0,"MULTIPOLYGON (((-122.24481 37.74894, -122.2445..."


In [60]:
water_cont_clean_2_unique.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [61]:
water_cont_clean_2_unique = water_cont_clean_2_unique.explode().reset_index(drop=True)

In [62]:
water_cont_clean_2_unique.head()

,geometry
0,"POLYGON ((-122.24481 37.74894, -122.24459 37.7..."
1,"POLYGON ((-122.20873 37.92157, -122.20865 37.9..."
2,"POLYGON ((-122.50616 37.92150, -122.50636 37.9..."
3,"POLYGON ((-122.26433 38.09357, -122.26374 38.0..."
4,"POLYGON ((-121.47137 36.99846, -121.47137 36.9..."


In [63]:
water_cont_clean_2_unique.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [64]:
water_cont_clean_2_unique.shape

(8091, 1)

In [66]:
green_unique.to_file('../data/outputs/greenspaces_sf_final.shp')
water_cont_clean_2_unique.to_file('../data/outputs/bluespaces_sf_final.shp')
water_clean_2.to_file('../data/outputs/ocean_sf_final.shp')